In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Get dates and gameid for each game in 2020/2021 English Premier League

url='https://api.sportsdata.io/v3/soccer/scores/json/CompetitionDetails/EPL'

response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "7e69e12a9f714caebb8c178cf904e445"})

result=pd.json_normalize(response.json())
games = pd.DataFrame(result['Games'][0])#.flatten()#.columns

games_final = games[['GameId','AwayTeamId','HomeTeamId','Day','AwayTeamName','HomeTeamName']]

Days=games_final['Day'].unique().tolist()

games_final['Day'] = pd.to_datetime(games_final['Day'])

TeamID = games_final['AwayTeamId'].unique().tolist()



In [3]:
url='https://api.sportsdata.io/v3/soccer/odds/json/GameOddsByDate/2020-09-12'

response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "7e69e12a9f714caebb8c178cf904e445"})

response=response.json()
result=pd.DataFrame(response)

In [4]:
#Selecting games only from EPL

result=result[result['RoundId']==602].reset_index()


#Selecting games which happened

result=result[result['Status']=='Final']

In [5]:
result

,index,GameId,RoundId,Season,SeasonType,Week,Day,DateTime,Status,AwayTeamId,...,AwayTeamName,HomeTeamName,GlobalGameId,GlobalAwayTeamId,GlobalHomeTeamId,HomeTeamScore,AwayTeamScore,TotalScore,PregameOdds,LiveOdds
2,6,32481,602,2021,1,1.0,2020-09-12T00:00:00,2020-09-12T11:30:00,Final,509,...,Arsenal FC,Fulham FC,90032481,90000509,90001462,0.0,6.0,6.0,"[{'GameOddId': 1178854, 'Sportsbook': 'Scrambl...",[]
3,9,32486,602,2021,1,1.0,2020-09-12T00:00:00,2020-09-12T14:00:00,Final,520,...,Southampton FC,Crystal Palace FC,90032486,90000520,90000512,2.0,0.0,2.0,"[{'GameOddId': 1178999, 'Sportsbook': 'Scrambl...",[]
4,190,32482,602,2021,1,1.0,2020-09-12T00:00:00,2020-09-12T16:30:00,Final,2148,...,Leeds United FC,Liverpool FC,90032482,90002148,90000515,8.0,6.0,14.0,"[{'GameOddId': 1179164, 'Sportsbook': 'Scrambl...",[]
5,196,32485,602,2021,1,1.0,2020-09-12T00:00:00,2020-09-12T19:00:00,Final,518,...,Newcastle United FC,West Ham United FC,90032485,90000518,90000527,0.0,4.0,4.0,"[{'GameOddId': 1179305, 'Sportsbook': 'Scrambl...",[]


In [6]:
#Aggregated Odds per game 

odds=result['PregameOdds']

empty_list = []

for i in range(len(odds)):
    
    try:
        
        empty_list.append(pd.DataFrame(odds[i]))
    
    except: 
        
        continue
        
games_odds=pd.concat(empty_list)

games=games_odds.groupby('GameId').agg({'HomeMoneyLine':'mean','AwayMoneyLine':'mean','DrawMoneyLine':'mean'})


In [7]:
#Merging with initial table

daily_games = pd.merge(left = result,
                                 right = games,
                                 how = 'inner', 
                                 left_on = "GameId", 
                                 right_on= "GameId")


daily_games=daily_games[['GameId', 'RoundId','Day', 
                         'AwayTeamId', 'HomeTeamId', 'AwayTeamName', 'HomeTeamName','HomeTeamScore',
                         'AwayTeamScore', 'TotalScore','HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine']]

daily_games[['Day']]=pd.to_datetime(daily_games['Day'])

In [8]:
#winner column
for index, row in daily_games.iterrows():
    if row['HomeTeamScore'] > row['AwayTeamScore']:
        #df.loc[index,'nova_coluna'] =  str(df.loc[index,'CODIGO'])  
        daily_games['Winner'] =  'Home'
    elif row['HomeTeamScore'] < row['AwayTeamScore']:
        daily_games['Winner'] =   'Away'
    else:
        daily_games['Winner'] =  'Tie'

In [21]:
def winner(row):
    if daily_games[row] > daily_games[row]:
        return 'Home'
    elif daily_games[row] < daily_games[row]:
        return  'Away'
    else:
        return 'Tie'

daily_games['Winner2'] = daily_games[['HomeTeamScore', 'AwayTeamScore']].apply(lambda x: winner(x), axis=1)


KeyError: "None of [Float64Index([0.0, 6.0], dtype='float64')] are in the [columns]"

In [9]:
daily_games

,GameId,RoundId,Day,AwayTeamId,HomeTeamId,AwayTeamName,HomeTeamName,HomeTeamScore,AwayTeamScore,TotalScore,HomeMoneyLine,AwayMoneyLine,DrawMoneyLine,Winner
0,32481,602,2020-09-12,509,1462,Arsenal FC,Fulham FC,0.0,6.0,6.0,625.666667,-212.833333,418.333333,Home
1,32486,602,2020-09-12,520,512,Southampton FC,Crystal Palace FC,2.0,0.0,2.0,298.333333,206.500000,319.833333,Home
